# 1) HHT using 'emd' package

In [ ]:
%matplotlib qt
import emd
# from PyEMD import EMD
import numpy as np
from scipy import ndimage

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define and simulate a simple signal
sample_rate = 512
seconds = 10
# peak_freq = 12
# noise_std = 0.5
# x = emd.utils.ar_simulate(peak_freq, sample_rate, seconds, noise_std=noise_std,
#                           random_seed=42, r=0.99)[:, 0]
# x *= 1e-4
x = np.load('/Users/creator/analysis/methods/HilbertTransform/exampledata.npy')
x = x[4000:9121]
x *= 1e-2
t = np.linspace(0, seconds, seconds * sample_rate)

# Plot the first 3 seconds of data
plt.figure(figsize=(10, 2))
plt.plot(t[:sample_rate*3], x[:sample_rate*3], 'k')
plt.show()

# Run a mask sift
imf = emd.sift.mask_sift(x, max_imfs=5)

emd.plotting.plot_imfs(imf[:2000, :], cmap=True, scale_y=True)

# # Compute frequency statistics
# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'nht')

# plt.figure(figsize=(8, 4))
# # Plot a simple histogram using frequency bins from 0-40 Hz
# plt.subplot(121)
# plt.hist(IF[:, 2], np.linspace(0, 40))
# plt.grid()
# plt.title('IF Histogram')
# plt.xticks(np.arange(0, 40, 4))
# plt.xlabel('Frequency (Hz)')
# # Plot an amplitude-weighted histogram using frequency bins from 0-40 Hz
# plt.subplot(122)
# plt.hist(IF[:, 2], np.linspace(0, 40), weights=IA[:, 2])
# plt.grid()
# plt.title('IF Histogram\nweighted by IA')
# plt.xticks(np.arange(0, 40, 4))
# plt.xlabel('Frequency (Hz)')
# plt.show()

# freq_edges, freq_centres = emd.spectra.define_hist_bins(0, 100, 128, 'linear')

# # Amplitude weighted HHT
# _, spec_weighted = emd.spectra.hilberthuang(IF, IA, freq_edges)

# # Unweighted HHT - we replace the instantaneous amplitude values with ones
# _, spec_unweighted = emd.spectra.hilberthuang(IF, np.ones_like(IA), freq_edges)

# plt.figure(figsize=(8, 4))
# plt.subplot(121)
# plt.plot(freq_centres, spec_unweighted)
# plt.xticks(np.arange(8)*12)
# plt.xlim([0, 100])
# plt.ylim([0, 6000])
# plt.xlabel('Frequency (Hz)')
# plt.title('Unweighted\nHilbert-Huang Transform')
# plt.subplot(122)
# plt.plot(freq_centres, spec_weighted)
# plt.xticks(np.arange(8)*12)
# plt.xlim([0, 100])
# plt.ylim([0, 6000])
# plt.xlabel('Frequency (Hz)')
# plt.title('IA-weighted\nHilbert-Huang Transform')
# plt.show()

# # Carrier frequency histogram definition
# freq_edges, freq_centres = emd.spectra.define_hist_bins(1, 25, 24, 'linear')

# _, hht = emd.spectra.hilberthuang(IF[:, 2], IA[:, 2], freq_edges, mode='amplitude')
# time_centres = np.arange(201) - 0.5

# plt.figure(figsize=(10, 8))
# # Add signal and IA
# plt.axes([0.1, 0.6, 0.64, 0.3])
# plt.plot(imf[:, 2], 'k')
# plt.plot(IA[:, 2], 'r')
# plt.legend(['IMF', 'IA'])
# plt.xlim([50, 150])

# # Add IF axis and legend
# plt.axes([0.1, 0.1, 0.8, 0.45])
# plt.plot(IF[:, 2], 'g', lw=3)
# plt.legend('IF')

# # Plot HHT
# # plt.pcolormesh(time_centres, freq_edges, hht, cmap='hot_r', vmin=0)
# plt.show()

# hht = ndimage.gaussian_filter(hht, 0.7)

# 2) HHT using 'scipy' and 'pyEMD' package

In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.signal import hilbert, chirp
from PyEMD import EMD

plt.rcParams['font.size'] = 14

In [2]:
def calculate_hilbert_spectrum(imfs, t, fs, n=5, plot_hilbert_spec=None, plot_inst_freq=None):
    '''
    Calculate hilbert amplitude spectrum from a given set of intrinsic mode functions.
    '''

    ## Create Hilbert spectrum
    T = t[-1] - t[0]; delta_t = 1 / fs
    fmin = fres = 1 / T; fmax = 1 / (n * delta_t)
    N = int(T / (n * delta_t))
    bin_centres = np.arange(N) * fres + fmin
    bin_edges = np.arange(N + 1) * fres + (fmin - fres / 2)

    hht = np.zeros((len(imfs), N, (len(t) - 2)))

    for j, imf in enumerate(imfs):
        Z = hilbert(imf)
        A = np.abs(Z)
        theta_inst = np.unwrap(np.angle(Z))
        f_inst = np.r_[np.nan,
                       0.5 * (np.angle(-Z[2:] * np.conj(Z[:-2])) + np.pi) / (2 * np.pi) * fs,
                       np.nan]
        t_spec = t[1:-1]; A_spec = A[1:-1]; f_spec = f_inst[1:-1]

        # Plot instantaneous frequency curves
        if plot_inst_freq and plot_inst_freq is not None:
            fig, (ax0, ax1) = plt.subplots(nrows=2)
            ax0.plot(t, imf, label='signal')
            ax0.plot(t, A, label='envelope')
            ax0.set_xlabel("time (s)")
            ax0.set_ylabel("signal (units)")
            ax0.legend()
            ax1.plot(t_spec, f_spec)
            ax1.set_xlabel("time (s)")
            ax1.set_ylabel("frequency (Hz)")
            fig.tight_layout()
            plt.show()

        # Binning of frequency values
        binned_freq = pd.cut(f_spec, bin_edges)
        bin_inds = binned_freq.codes

        # Populate Hilbert spectrum matrix
        for i, bin_ind in enumerate(bin_inds):
            if bin_ind > 0:
                hht[j][bin_ind][i] = A_spec[i]
            else:
                pass

    hht_sum = np.sum(hht, axis=0)
    
    # Plot Hilbert spectrum for all IMFs
    if plot_hilbert_spec and plot_hilbert_spec is not None:
        plt.figure()
        plt.pcolormesh(t_spec, bin_centres, hht_sum)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.show()
        
    return hht_sum

def derivative_central_secondorder(x, fs):
    n = len(x)
    dxdt = np.zeros((n, 1))
    dxdt[0] = (x[1] - x[0]) * fs # 1 / dt = fs
    for i in range(1, n-1):
        dxdt[i] = (x[i+1] - x[i-1]) / 2 * fs
    dxdt[n-1] = (x[n-1] - x[n-2]) * fs
    return dxdt

def derivative_central_fourthorder(x, fs):
    n = len(x)
    dxdt = np.zeros((n, 1))
    dxdt[0] = (x[1] - x[0]) * fs # 1 / dt = fs
    dxdt[1] = (x[2] - x[0]) * fs / 2
    for i in range(2, n-2):
        dxdt[i] = (- x[i+2] + 8*x[i+1] - 8*x[i-1] + x[i-2]) * fs / 12
    dxdt[n-2] = (x[n-1] - x[n-3]) * fs / 2
    dxdt[n-1] = (x[n-1] - x[n-2]) * fs
    return dxdt

In [7]:
# Import data
fs = 1000
x = np.load('./exampledata.npy')
x = x[7000:9001]
t = np.arange(0, len(x)/fs, 1/fs)
# t = np.arange(0, 2001/fs, 1/fs); x = np.sin(2*np.pi*3*t) + np.sin(2*np.pi*6*t);

In [8]:
# EMD
emd_decomp = EMD()
imfs = emd_decomp(x)
C = imfs[:-1]

In [9]:
# Visualize EMD
plt.figure(figsize=(12, 12))
for i in range(len(imfs)-1):
    plt.subplot(len(imfs)+1, 1, i+1)
    plt.plot(t, x, color='0.8')
    plt.plot(t, imfs[i], 'k')
    plt.xlim([np.min(t), np.max(t)])
    plt.ylabel('IMF ' + str(i + 1))
plt.subplot(len(imfs)+1, 1, i+2)
plt.plot(t, x, color='0.8')
plt.plot(t, imfs[-1], 'k')
plt.xlim([np.min(t), np.max(t)])
plt.ylabel('Residual')
plt.xlabel('Time (s)')
plt.tight_layout()
plt.show()

In [10]:
hht = calculate_hilbert_spectrum(C, t, fs, plot_hilbert_spec=True)

In [ ]:
# HT on IMF-3 (i = 2)
signal_x = imfs[2]
signal_z = hilbert(signal_x)
amp = np.abs(signal_z)
inst_phase = np.unwrap(np.angle(signal_z))
# inst_freq = np.diff(inst_phase) / (2 * np.pi) * fs
# inst_freq = derivative_central_secondorder(inst_phase, fs) / (2 * np.pi)
# inst_freq = derivative_central_fourthorder(inst_phase, fs) / (2 * np.pi)
inst_freq = np.r_[np.nan, 0.5*(np.angle(-signal_z[2:]*np.conj(signal_z[:-2]))+np.pi)/(2*np.pi)*fs, np.nan]

In [ ]:
t_hht = t[1:-1]; amp_hht = amp[1:-1]; f_hht = inst_freq[1:-1]

In [ ]:
# Visualize HT
fig, (ax0, ax1) = plt.subplots(nrows=2)
ax0.plot(t, signal_x, label='signal')
ax0.plot(t, amp, label='envelope')
ax0.set_xlabel("time (s)")
ax0.set_ylabel("signal (units)")
# ax0.legend()
ax1.plot(t_hht, f_hht)
ax1.set_xlabel("time (s)")
ax1.set_ylabel("frequency (Hz)")
fig.tight_layout()
plt.show()

In [ ]:
# Create Hilbert spectrum
n = 5
T = t[-1] - t[0]
delta_t = 1 / fs
fmin = 1 / T
fres = 1 / T
fmax = 1 / (n * delta_t)
N = int(T / (n * delta_t))
print(delta_t)
print(fmin)
print(fres)
print(fmax)
print(N)

In [ ]:
bin_centres = np.arange(N) * fres + fmin
bin_edges = np.arange(N + 1) * fres + (fmin - fres / 2)
binned_freq = pd.cut(f_hht, bin_edges)

In [ ]:
f_hht[0]

In [ ]:
binned_freq[0]

In [ ]:
bin_inds = binned_freq.codes
bin_inds

In [ ]:
bin_wins = binned_freq.categories
bin_wins[18]

In [ ]:
len(bin_inds)

In [ ]:
hht = np.zeros((N, len(bin_inds)))
for i, bin_ind in enumerate(bin_inds):
    if bin_ind > 0:
        hht[bin_ind][i] = amp_hht[i]
    else:
        pass

In [ ]:
plt.figure()
plt.pcolormesh(t_hht, bin_centres, hht)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()